In [73]:
import numpy as np
import pandas as pd
import seaborn as sns

C:\Anaconda3\lib\site-packages\pandas\core\computation\__init__.py:18: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


# Get Data

The first analysis is meant to dive in to crypto currencies trading, therefor the most know currencies are chosen.
The main idea is to have a look how those currencies behaved in the time after they reached their popularity.

**Chosen Currencies:**
- BTC
- ETH
- LTC
- NEO

In [1]:
import urllib.request
import json

def retrieve(url):
    with urllib.request.urlopen(url) as response:
        html = response.read().decode("utf-8")
        data = json.loads(html)
    return data

In [81]:
import time
import datetime


def timestampToDatetime(timestamp):
    return datetime.datetime.fromtimestamp(timestamp)


def strTimeToDatetime(strTime):
    return datetime.datetime.strptime(strTime, "%Y-%m-%d %H:%M:%S")


def strTimeToTimestamp(strTime):
    return strTimeToDatetime(strTime).timestamp()

In [64]:
def queryPoloniex(beginDatetime, endDatetime, currencyPair, period):
    beginTimestamp, endTimestamp = beginDatetime.timestamp(), endDatetime.timestamp()
    url = 'https://poloniex.com/public?command=returnTradeHistory&currencyPair={pair}&start={begin}&end={end}'
    url = 'https://poloniex.com/public?command=returnChartData&currencyPair={pair}&start={begin}&end={end}&period={period}'
    return retrieve(url.format(pair=currencyPair, begin=beginTimestamp, end=endTimestamp, period=period))

In [71]:
from dateutil.rrule import rrule, MONTHLY 
from time import sleep

def retrieveLongWindowPoloniex(beginTime, endTime, currencyPair, period):
    beginDT = strTimeToDatetime(beginTime)
    endDT = strTimeToDatetime(endTime)
    data = queryPoloniex(beginDT, endDT, currencyPair, period)
    return data
        

In [77]:
def renameDateToTimestamp(df):
    return df.rename(columns={'date': 'timestamp'})

## BTC - USD

In [68]:
btc_usd = retrieveLongWindowPoloniex('2014-01-01 00:00:00', '2017-11-28 23:59:59', 'USDT_BTC', 900)

In [82]:
btcUsdDf = pd.DataFrame(btc_usd)
btcUsdDf = renameDateToTimestamp(btcUsdDf)
btcUsdDf['date'] = btcUsdDf['timestamp'].apply(timestampToDatetime)

In [84]:
btcUsdDfTimeSelected = btcUsdDf[btcUsdDf['date'] > '2016-01-01 00:00:00']

In [86]:
btcUsdDfTimeSelected.to_csv('./data/USDT-BTC_15min_2016-01-01_2017-11-28.tsv', index=False, sep='\t')